In [4]:
%load_ext autoreload
%autoreload 2
import bootstrap

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from mxnet import ndarray as nd
import numpy as np
from dataclasses import dataclass

In [ ]:
# class BAM:
    
    
#     def __init__(n_inputs, n_outputs):
#         self._
        
    
    

In [ ]:
# step 1: replicate basic experiments from knoblauch chapter


In [ ]:
# define weight arrays
# later on make them sparse


In [63]:
n_u = 10  # input neurons
n_v = 10  # output neurons


u = nd.zeros([n_u])
v = nd.zeros([n_v])

W = nd.zeros([n_v, n_u])

In [64]:
W_potential = W.copy()

In [65]:
W_potential


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
<NDArray 10x10 @cpu(0)>

In [66]:
W_potential[:,2] = list(range(10))
W_potential[:,5] = list(range(10))

In [72]:
W_potential


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 3. 0. 0. 3. 0. 0. 0. 0.]
 [0. 0. 4. 0. 0. 4. 0. 0. 0. 0.]
 [0. 0. 5. 0. 0. 5. 0. 0. 0. 0.]
 [0. 0. 6. 0. 0. 6. 0. 0. 0. 0.]
 [0. 0. 7. 0. 0. 7. 0. 0. 0. 0.]
 [0. 0. 8. 0. 0. 8. 0. 0. 0. 0.]
 [0. 0. 9. 0. 0. 9. 0. 0. 0. 0.]]
<NDArray 10x10 @cpu(0)>

In [83]:
W_potential.topk(axis=1, ret_typ='value', k=1, is_ascend=False)


[[0.]
 [1.]
 [2.]
 [3.]
 [4.]
 [5.]
 [6.]
 [7.]
 [8.]
 [9.]]
<NDArray 10x1 @cpu(0)>

In [84]:
W_potential.topk(axis=1, ret_typ='indices', k=1, is_ascend=False)


[[0.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]]
<NDArray 10x1 @cpu(0)>

In [91]:
W_potential.topk(axis=1, ret_typ='mask', k=2, is_ascend=False)


[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]]
<NDArray 10x10 @cpu(0)>

In [ ]:
# TEST

